## XGBoost 


### Boosting


* Not a specific machine learning algorithm
* Concept that can be applied to a set of machine learning models
"Meta-algorithm"
* Ensemble meta-algorithm used to convert many weak learners into a strong learner

**weak Learner** - Learners which are slightly better than randomness eg. Decision tree with accuracy greater than 50%

**How boosting works?**

* Iteratively learning a set of weak models on subsets of the data 
* Weighing each weak prediction according to each weak learner's performance
* Combine the weighted predictions to obtain a single weighted prediction

... that is much better than the individual predictions themselves!







What's the difference from Random Forrest?





In [ ]:
#how to install 
#pip install xgboost



In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np 
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
# load the dataset 
iris = datasets.load_iris()

X,y = iris.data , iris.target

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train,y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy:  ", (accuracy))

**When to NOT use XGBoost?**

* Image recognition
* Computer vision
* Natural language processing and understanding problems
* When the number of training samples is significantly smaller than the number of features

### Objective Functions and Base Learners

**Objective Functions**

* Quantifies how far off a prediction is from the actual result
* Measures the difference between estimated and true values for some collection of data
* Goal: Find the model that yields the minimum value of the loss function

#### Common Loss Functions and XGBoost
Loss function names in xgboost:
* **reg:linear** - use for regression problems
* **reg:logistic** - use when you want probability rather than just decision
* **binary:logistic** - use for classification problems when you want just decision, not probability
    

**Base Learners and Why We Need Them?**

* XGBoost involves creating a meta-model that is composed of many individual models that combine to give a final prediction
* Individual models = base learners
* Want base learners that when combined create final prediction that is non-linear
* Each base learner should be good at distinguishing or predicting different parts of the dataset
* Two kinds of base learners: **tree** and **linear**

#### Trees as Base Learners in Scikit API

In [ ]:
boston_data = pd.read_csv("../datasets/boston_housing.csv")
X, y = boston_data.iloc[:,:-1],boston_data.iloc[:,-1] 
X_train, X_test, y_train, y_test= train_test_split(X, y,
test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective='reg:linear',
n_estimators=10, seed=123)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))

####  Linear Base Learner in XGBoost API

In [ ]:
boston_data = pd.read_csv("../datasets/boston_housing.csv")
X, y = boston_data.iloc[:,:-1],boston_data.iloc[:,-1] 

X_train, X_test, y_train, y_test= train_test_split(X, y,
test_size=0.2, random_state=123)
DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg = xgb.train(params = params, dtrain=DM_train,
num_boost_round=10)
preds = xg_reg.predict(DM_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))

### Tuning the Model

Why tune? because it gives better accuracy

**Untuned Model**


In [ ]:

housing_data = pd.read_csv("../datasets/ames_housing_trimmed_processed.csv")

X,y = housing_data[housing_data.columns.tolist()[:-1]], housing_data[housing_data.columns.tolist()[-1]]

housing_dmatrix = xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, params=untuned_params, nfold=4, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_results_rmse))

print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

In [ ]:
housing_data = pd.read_csv("../datasets/ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]], housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X,label=y)
tuned_params = {"objective":"reg:linear",'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 5}
tuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, params=tuned_params, nfold=4, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)
print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

#### Common tree tunable parameters

* **learning rate:** learning rate/eta
* **gamma:** min loss reduction to create new tree split
* **lambda:** L2 reg on leaf weights
* **alpha:** L1 reg on leaf weights
* **max_depth:** max depth per tree
* **subsample:** % samples used per tree 
* **colsample_bytree:** % features used per tree

## Assignment

1. Load the kidney disease dataset 
2. Try to predict the disease given blood values 

**Main Challenges : Many missing values**

References :

https://xgboost.readthedocs.io/en/latest/model.html
https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/

Kidney diseases dataset details :https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

## XGBoost Part 2

Data Preprocessing Step 

We will again look at the AMES housing dataset 



In [2]:
import pandas as pd
df = pd.read_csv('../datasets/ames_unprocessed_data.csv')
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,...,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageArea,PavedDrive,SalePrice
0,60,RL,65.0,8450,CollgCr,1Fam,2Story,7,5,2003,...,1710,1,0,2,1,3,0,548,Y,208500
1,20,RL,80.0,9600,Veenker,1Fam,1Story,6,8,1976,...,1262,0,1,2,0,3,1,460,Y,181500
2,60,RL,68.0,11250,CollgCr,1Fam,2Story,7,5,2001,...,1786,1,0,2,1,3,1,608,Y,223500
3,70,RL,60.0,9550,Crawfor,1Fam,2Story,7,5,1915,...,1717,1,0,1,0,3,1,642,Y,140000
4,60,RL,84.0,14260,NoRidge,1Fam,2Story,8,5,2000,...,2198,1,0,2,1,4,1,836,Y,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 21 columns):
MSSubClass      1460 non-null int64
MSZoning        1460 non-null object
LotFrontage     1201 non-null float64
LotArea         1460 non-null int64
Neighborhood    1460 non-null object
BldgType        1460 non-null object
HouseStyle      1460 non-null object
OverallQual     1460 non-null int64
OverallCond     1460 non-null int64
YearBuilt       1460 non-null int64
Remodeled       1460 non-null int64
GrLivArea       1460 non-null int64
BsmtFullBath    1460 non-null int64
BsmtHalfBath    1460 non-null int64
FullBath        1460 non-null int64
HalfBath        1460 non-null int64
BedroomAbvGr    1460 non-null int64
Fireplaces      1460 non-null int64
GarageArea      1460 non-null int64
PavedDrive      1460 non-null object
SalePrice       1460 non-null int64
dtypes: float64(1), int64(15), object(5)
memory usage: 239.6+ KB


The data has five categorical columns: MSZoning, PavedDrive, Neighborhood, BldgType, and HouseStyle. Scikit-learn has a LabelEncoder function that converts the values in each categorical column into integers. You'll practice using this here.

In [5]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == 'object')

# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()

# Print the head of the categorical columns
print(df[categorical_columns].head())

# Create LabelEncoder object: le
le = LabelEncoder()

# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

# Print the head of the LabelEncoded categorical columns
print(df[categorical_columns].head())
# print()

  MSZoning Neighborhood BldgType HouseStyle PavedDrive
0       RL      CollgCr     1Fam     2Story          Y
1       RL      Veenker     1Fam     1Story          Y
2       RL      CollgCr     1Fam     2Story          Y
3       RL      Crawfor     1Fam     2Story          Y
4       RL      NoRidge     1Fam     2Story          Y
   MSZoning  Neighborhood  BldgType  HouseStyle  PavedDrive
0         3             5         0           5           2
1         3            24         0           2           2
2         3             5         0           5           2
3         3             6         0           5           2
4         3            15         0           5           2


In [ ]:
categorical_mask

**We got the categorical columns in one go but is it enough?**



What about the neighbourhood values?

In the categorical columns of this dataset, there is no natural ordering between the entries. As an example: Using LabelEncoder, the CollgCr Neighborhood was encoded as 5, while the Veenker Neighborhood was encoded as 24, and Crawfor as 6. Is Veenker "greater" than Crawfor and CollgCr? No - and allowing the model to assume this natural ordering may result in poor performance.

In [9]:
# Import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create OneHotEncoder: ohe
ohe = OneHotEncoder(categorical_features=categorical_mask, sparse=False)

# Apply OneHotEncoder to categorical columns - output is no longer a dataframe: df_encoded
df_encoded = ohe.fit_transform(df)

# Print first 5 rows of the resulting dataset - again, this will no longer be a pandas dataframe
# print(df_encoded[:5, :])
print(pd.DataFrame(df_encoded))

# Print the shape of the original DataFrame
print(df.shape)

# Print the shape of the transformed array
print(df_encoded.shape)


       0    1    2    3    4    5    6    7    8    9     ...      52      53  \
0     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     0.0  1710.0   
1     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     0.0  1262.0   
2     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     1.0  1786.0   
3     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     1.0  1717.0   
4     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     0.0  2198.0   
5     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     1.0  1362.0   
6     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     1.0  1694.0   
7     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     0.0  2090.0   
8     0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0    ...     1.0  1774.0   
9     0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0    ...     1.0  1077.0   
10    0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0    ...     0.0  1040.0   
11    0.0  0.0  0.0  1.0  0.

Using a DictVectorizer on a DataFrame that has been converted to a dictionary allows you to get label encoding as well as one-hot encoding in one go.

In [14]:
# Import DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# Convert df into a dictionary: df_dict
df_dict = df.to_dict("records")
print(df_dict)

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse="False")

# Apply dv on df: df_encoded
df_encoded = dv.fit_transform(df_dict)

# Print the resulting first five rows
print(df_encoded[:5,:])

# Print the vocabulary
print(dv.vocabulary_)


[{'LotArea': 8450.0, 'Remodeled': 0.0, 'BsmtHalfBath': 0.0, 'BsmtFullBath': 1.0, 'MSSubClass': 60.0, 'YearBuilt': 2003.0, 'FullBath': 2.0, 'HalfBath': 1.0, 'MSZoning': 3.0, 'LotFrontage': 65.0, 'GrLivArea': 1710.0, 'PavedDrive': 2.0, 'HouseStyle': 5.0, 'Neighborhood': 5.0, 'BedroomAbvGr': 3.0, 'Fireplaces': 0.0, 'OverallQual': 7.0, 'SalePrice': 208500.0, 'OverallCond': 5.0, 'GarageArea': 548.0, 'BldgType': 0.0}, {'LotArea': 9600.0, 'Remodeled': 0.0, 'BsmtHalfBath': 1.0, 'BsmtFullBath': 0.0, 'MSSubClass': 20.0, 'YearBuilt': 1976.0, 'FullBath': 2.0, 'HalfBath': 0.0, 'MSZoning': 3.0, 'LotFrontage': 80.0, 'GrLivArea': 1262.0, 'PavedDrive': 2.0, 'HouseStyle': 2.0, 'Neighborhood': 24.0, 'BedroomAbvGr': 3.0, 'Fireplaces': 1.0, 'OverallQual': 6.0, 'SalePrice': 181500.0, 'OverallCond': 8.0, 'GarageArea': 460.0, 'BldgType': 0.0}, {'LotArea': 11250.0, 'Remodeled': 1.0, 'BsmtHalfBath': 0.0, 'BsmtFullBath': 1.0, 'MSSubClass': 60.0, 'YearBuilt': 2001.0, 'FullBath': 2.0, 'HalfBath': 1.0, 'MSZoning': 

### Implementing Pipelines

In [21]:
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression

steps = [
    ('le',LabelEncoder()),
    ('ohe',OneHotEncoder(sparse=False, categorical_features=categorical_mask)),
    ('clf',LinearRegression())
]

pipeline = Pipeline(steps)

final_data = df.drop("SalePrice",axis=1)
final_label = df["SalePrice"]

print(final_label.head())

pipeline.fit(final_data, final_label)


0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


TypeError: fit_transform() takes 2 positional arguments but 3 were given

## Kidney disease dataset 

The chronic kidney disease dataset contains both categorical and numeric features, but contains lots of missing values. The goal here is to predict who has chronic kidney disease given various blood indicators as features.

**sklearn_pandas**  allows you to chain many more processing steps inside of a pipeline than are currently supported in scikit-learn. Specifically, you'll be able to impute missing categorical values directly using the Categorical_Imputer() class in sklearn_pandas, and the DataFrameMapper() class to apply any arbitrary sklearn-compatible transformer on DataFrame columns, where the resulting output can be either a NumPy array or DataFrame.

The mapper takes a list of tuples. The first is a column name from the pandas DataFrame, or a list containing one or multiple columns (we will see an example with multiple columns later). The second is an object which will perform the transformation which will be applied to that column.

In [ ]:
# Import necessary modules
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import Imputer

data = pd.read_csv('../datasets/chronic_kidney_with_class.csv')
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature], Imputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
                                                [(category_feature, CategoricalImputer()) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

#### Feature Union

Concatenates results of multiple transformer objects.
This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

In [ ]:
# Import FeatureUnion
from sklearn.pipeline import FeatureUnion

# Combine the numeric and categorical transformations
numeric_categorical_union = FeatureUnion([
                                          ("num_,mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])

In [ ]:
##One last step before jumping 
## Don't understand at the moment leave it
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict
import warnings
import pandas as pd
 
class Dictifier(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
 
    def transform(self, X):
        if type(X) == pd.core.frame.DataFrame:
            return X.to_dict("records")
        else:
            return pd.DataFrame(X).to_dict("records")
 
    def fit_transform(self, X, y=None, **fit_params):
        """Fit to data, then transform it.
 
        Fits transformer to X and y with optional parameters fit_params
        and returns a transformed version of X.
 
        Parameters
        ----------
        X : numpy array of shape [n_samples, n_features]
            Training set.
 
        y : numpy array of shape [n_samples]
            Target values.
 
        Returns
        -------
        X_new : numpy array of shape [n_samples, n_features_new]
            Transformed array.
 
        """
        # non-optimized default implementation; override when a better
        # method is possible for a given clustering algorithm
        if y is None:
            # fit method of arity 1 (unsupervised transformation)
            return self.fit(X, **fit_params).transform(X)
        else:
            # fit method of arity 2 (supervised transformation)
            return self.fit(X, y, **fit_params).transform(X)
 
    def fit(self, X, y=None):
        return self
 
    def get_params(self, deep=True):
        """Get parameters for this estimator.
 
        Parameters
        ----------
        deep : boolean, optional
            If True, will return the parameters for this estimator and
            contained subobjects that are estimators.
 
        Returns
        -------
        params : mapping of string to any
            Parameter names mapped to their values.
        """
        out = dict()
        for key in self._get_param_names():
            # We need deprecation warnings to always be on in order to
            # catch deprecated param values.
            # This is set in utils/__init__.py but it gets overwritten
            # when running under python3 somehow.
            warnings.simplefilter("always", DeprecationWarning)
            try:
                with warnings.catch_warnings(record=True) as w:
                    value = getattr(self, key, None)
                if len(w) and w[0].category == DeprecationWarning:
                    # if the parameter is deprecated, don't show it
                    continue
            finally:
                warnings.filters.pop(0)
 
            # XXX: should we rather test if instance of estimator?
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)
            out[key] = value
        return out
 
    def set_params(self, **params):
        """Set the parameters of this estimator.
 
        The method works on simple estimators as well as on nested objects
        (such as pipelines). The latter have parameters of the form
        ``<component>__<parameter>`` so that it's possible to update each
        component of a nested object.
 
        Returns
        -------
        self
        """
        if not params:
            # Simple optimization to gain speed (inspect is slow)
            return self
        valid_params = self.get_params(deep=True)
 
        nested_params = defaultdict(dict)  # grouped by prefix
        for key, value in params.items():
            key, delim, sub_key = key.partition('__')
            if key not in valid_params:
                raise ValueError('Invalid parameter %s for estimator %s. '
                                 'Check the list of available parameters '
                                 'with `estimator.get_params().keys()`.' %
                                 (key, self))
 
            if delim:
                nested_params[key][sub_key] = value
            else:
                setattr(self, key, value)
 
        for key, sub_params in nested_params.items():
            valid_params[key].set_params(**sub_params)
 
        return self